# Использование библиотек Python для визуализации. Библиотека `Altair 5.0`

Для загрузки данных из Google-документов можно пользоваться библиотекой `gspread`.

Данный код будет работать, если у вас установлена библиотека `gspread` как минимум версии 5.4.7 и вы имееете ключ для сервисного аккаунта в формате JSON. Как получить ключ, можно ознакомиться в документации [по ссылке](https://support.google.com/a/answer/7378726?hl=ru).

In [ ]:
import gspread
import pandas as pd

gc = gspread.service_account('service_account_key.json')
wks = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1NObIGEC9ZxjOSs44KHLn-ftbsxdFHGVKB8Z-KzjPzYs/edit?usp=sharing'
).worksheet(
    'Orders'
)

superstore_data = pd.DataFrame(wks.get_all_records())

superstore_data = superstore_data.set_index('Row ID')
superstore_data['Order Date'] = pd.to_datetime(superstore_data['Order Date'])
superstore_data['Ship Date'] = pd.to_datetime(superstore_data['Ship Date'])
superstore_data.head()

Используйте следующий код для загрузки данных в ноутбук, если у вас не получилось воспользоваться библиотекой `gspread`.

Для корректной работы предварительно файл `superstore.csv` необходимо загрузить в Google Collab. Файл `superstore.csv` вы найдете в дополнительных материалах вместе с данным ноутбуком.

In [2]:
import pandas as pd

superstore_data = pd.read_csv('superstore.csv', delimiter=";")
superstore_data = superstore_data.set_index('Row ID')
superstore_data['Order Date'] = pd.to_datetime(superstore_data['Order Date'])
superstore_data['Ship Date'] = pd.to_datetime(superstore_data['Ship Date'])
superstore_data.head()

C:\Users\daini\AppData\Local\Temp\ipykernel_28460\2236403760.py:5: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  superstore_data['Order Date'] = pd.to_datetime(superstore_data['Order Date'])
C:\Users\daini\AppData\Local\Temp\ipykernel_28460\2236403760.py:6: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  superstore_data['Ship Date'] = pd.to_datetime(superstore_data['Ship Date'])


,Order ID,Order Date,Order Priority,Order Quantity,Sales,Discount,Ship Mode,Profit,Unit Price,Shipping Cost,Customer Name,Province,Region,Customer Segment,Product Category,Product Sub-Category,Product Name,Product Container,Ship Date
Row ID,,,,,,,,,,,,,,,,,,,
1,3,2010-10-13,Low,6,261.5400,0.04,Regular Air,-213.25,38.94,35.00,Muhammed MacIntyre,Nunavut,Nunavut,Small Business,Office Supplies,Storage & Organization,"Eldon Base for stackable storage shelf, platinum",Large Box,2010-10-20
2,6,2012-02-20,Not Specified,2,6.9300,0.01,Regular Air,-4.64,2.08,2.56,Ruben Dartt,Alberta,West,Corporate,Office Supplies,"Scissors, Rulers and Trimmers",Kleencut® Forged Office Shears by Acme United ...,Small Pack,2012-02-21
3,32,2011-07-15,High,26,2808.0800,0.07,Regular Air,1054.82,107.53,5.81,Liz Pelletier,Alberta,West,Corporate,Furniture,Office Furnishings,Tenex Contemporary Contur Chairmats for Low an...,Medium Box,2011-07-17
4,32,2011-07-15,High,24,1761.4000,0.09,Delivery Truck,-1748.56,70.89,89.30,Liz Pelletier,Alberta,West,Corporate,Furniture,Tables,KI Conference Tables,Jumbo Box,2011-07-16
5,32,2011-07-15,High,23,160.2335,0.04,Regular Air,-85.13,7.99,5.03,Liz Pelletier,Alberta,West,Corporate,Technology,Telephones and Communication,Bell Sonecor JB700 Caller ID,Medium Box,2011-07-17


Посмотрим, какие типы данных у нас получились:

In [3]:
superstore_data.dtypes

Order ID                         int64
Order Date              datetime64[ns]
Order Priority                  object
Order Quantity                   int64
Sales                          float64
Discount                       float64
Ship Mode                       object
Profit                         float64
Unit Price                     float64
Shipping Cost                  float64
Customer Name                   object
Province                        object
Region                          object
Customer Segment                object
Product Category                object
Product Sub-Category            object
Product Name                    object
Product Container               object
Ship Date               datetime64[ns]
dtype: object

## Начало работы с библиотекой `Altair`. Построение scatter plot

Импортируем библиотеку `Altair` и выполним предварительные настройки для работы:

In [4]:
# В этой версии ноутбука используется версия 5.0 библиотеки altair
# Синтаксис версии 4.x отличается
import altair as alt
alt.themes.enable('dark')
alt.data_transformers.disable_max_rows()
alt.warnings.simplefilter(action='ignore')

# Если графики не отображаются, попробуйте поменять рендерер на html
alt.renderers.enable("mimetype");
# alt.renderers.enable("html");

Построим диаграмму рассеяния — график корреляции `Sales` и `Profit`.

*   В примере мы выставили горизонтальную и вертикальную координаты, а также добавили поле `Customer Segment` как поле данных, по которому выбирается цвет точек.

*   Функция `interactive()` делает диаграмму интерактивной: по колесику мышки мы можем увеличивать или уменьшать график, курсором можно по графику перемещаться.

*   В примере настроена интерактивность легенды графика — мы можем смотреть отдельные категории. В `Altair` мы создали выделение — интерактивный элемент. Далее созданное выделение `selection` привязали к графику: мы создали в примере мультивыбор, чтобы пользователь мог выбирать сразу несколько сегментов. В параметрах указано, по каким полям будет выполняться фильтрация — `Customer Segment`. С помощью параметра `bind` выбираем, что выделение будет привязано к легенде. Мы будем менять прозрачность точек, воспользовавшись инструментом библиотеки `Altair` — `alt.condition`.



In [5]:
selection = alt.selection_multi(fields=['Customer Segment'], bind='legend')

chart = alt.Chart(
    superstore_data[['Sales','Profit','Customer Segment']]
).mark_point(
).add_params(
    selection
).encode(
    x='Sales',
    y='Profit',
    color='Customer Segment',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).interactive()

# График можно сохранить в формате json для вставки в веб-страницу или просмотра в Vega Editor: https://vega.github.io/editor/
# with open('chart.json', 'w') as f:
#     f.write(chart.to_json())
# chart.to_json()
chart

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


## Построение столбчатой диаграммы

Создадим еще один график — столбчатую диаграмму распределения продаж по дням. 
*   Мы будем использовать функцию `day()`, чтобы получить день недели, и функцию `mean()` для получения среднего значения продаж.
*   Для оси x мы прописали тип поля `Order Date` — ординальное.
*   Ширина графика задана через свойство `properties`.
*   В примере использован специальный класс — `alt.Y`, указав, что будет выбираться кастомная ось (таким образом мы убрали горизонтальные линии сетки).

In [6]:
chart = alt.Chart(
    superstore_data[['Order Date', 'Sales', 'Customer Segment']]
).mark_bar(
).encode(
    x='day(Order Date):O',
    y=alt.Y('mean(Sales)', axis=alt.Axis(grid=False)),
    color='Customer Segment'
).properties(
    width=500
)

chart

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
